In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import sys
sys.path.append('../')

import torch
from torch import nn
from torch import optim

if torch.cuda.is_available(): torch.cuda.manual_seed_all(3)
torch.cuda.is_available()
torch.cuda.get_device_name(0)

pathDataset = './Datasets/'
pathModel = './Models/BERT_DT.pt'

# 1. Import Dataset

In [2]:
import pandas as pd
data = pd.read_csv(pathDataset + 'test.csv', header=None)
data.head()

,0,1
0,1,I got 'new' tires from them and within two wee...
1,1,Don't waste your time. We had two different p...
2,1,All I can say is the worst! We were the only 2...
3,1,I have been to this restaurant twice and was d...
4,1,Food was NOT GOOD at all! My husband & I ate h...


# 2. Load the model

Il modello KerMIT è formato da PyDTK e Bert ...

In [3]:
from kerMIT.samples.models import DTBert

BERT_DIM = 768
TREE_DIM = 4000
OUTPUT_DIM = 5

model = DTBert(BERT_DIM, TREE_DIM, OUTPUT_DIM) 
model.cuda()

criterion = nn.NLLLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-5)

model = torch.load(pathModel)
print(model)

DTBert(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
     

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'transformers.modeling_bert.BertModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'transformers.modeling_bert.BertEmbeddings' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'transformers.modeling_bert.BertEncoder' has changed. you can retrieve the original source code by accessing the object's source

# 3. Load PyDTK

In [4]:
from kerMIT.dtk import DT
from kerMIT.operation import fast_shuffled_convolution
from kerMIT.explain import activationSubtreeLRP as act
from kerMIT.explain import modelToExplain as mte
from kerMIT.explain import kerMITviz 

calculator = DT(dimension=4000, LAMBDA=0.4, operation=fast_shuffled_convolution)

# 4. Select an example from the dataset

Per selezionare un esempio del dataset bisogna cambiare il valore $index$

In [5]:
from kerMIT.samples import utils

index = 33328
sentence = data.loc[index][1]
target = data.loc[index][0]

tree_sentence, dtk_sentence, bert_sentence = utils.get_sentence(sentence, calculator)

# 5. Compute Layer-wise Relevance Propagation

In [6]:
from  kerMIT.explain_pytorch import LRP_linear_layer as LRP_t

y_predict = model.get_activation(bert_sentence, dtk_sentence).cpu()
hin, w, b, hout, Rout, bias_nb_units, eps, bias_factor = LRP_t.prepare_input_LRP(y_predict, dtk_sentence, model, BERT_DIM)
Rin = LRP_t.lrp_linear_torch(hin.cpu(), w.cpu(), b.cpu(), hout.cpu(), Rout.cpu(), bias_nb_units, eps, bias_factor, debug=False)

act_lrp = act.ActivationSubtreeLRP(calculator)
act_tree_lrp = act_lrp.activationQC(Rin.detach().numpy(), tree_sentence)

# 6. Show the relevance of the syntactic tree

In [7]:
from tree_encode import parse as parse_tree

heat_parse_tree = kerMITviz.assign_contribution_nodes(act_tree_lrp)
kerMITviz.show_kerMITviz(heat_parse_tree)

100%|██████████| 1/1 [00:00<00:00, 336.86it/s]
